In [1]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification
import torch
from datasets import load_dataset
import pandas as pd
import os
import sys
from torchvision import transforms
from torch.utils.data import DataLoader
from pytorch_lightning import LightningModule
from torchmetrics.classification import MultilabelF1Score
from torchmetrics import AUROC
import torch
import onnx
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

2024-05-02 09:53:10.937541: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-02 09:53:16.069905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 09:53:16.070201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 09:53:16.083454: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 09:53:17.890833: I tensorflow/core/platform/cpu_feature_g

In [2]:
notebook_dir = os.getcwd()
project_dir = os.path.dirname(notebook_dir)

# Add the src directory to sys.path
src_dir = os.path.join(project_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from utils import generate_class_weights, show_batch_images
from data import ChestXray14HFDataset
from trainers import MultiLabelLightningModule

## load dataframes

In [3]:
data_path = "/cluster/home/taheeraa/datasets/chestxray-14"

In [4]:
labels = [
    "Atelectasis", 
    "Cardiomegaly",
    "Effusion", 
    "Infiltration", 
    "Mass",
    "Nodule",
    "Pneumonia",
    "Pneumothorax",  
    "Consolidation",
    "Edema",
    "Emphysema",
    "Fibrosis",
    "Pleural_Thickening",
    "Hernia"
]
file_path_train = data_path + '/train_official.txt'
file_path_val = data_path + '/val_official.txt'
file_path_test = data_path + '/test_official.txt'

columns = ['Image Filename'] + labels

df_train = pd.read_csv(file_path_train, sep='\s+', names=columns)
df_val = pd.read_csv(file_path_val, sep='\s+', names=columns)
df_test = pd.read_csv(file_path_test, sep='\s+', names=columns)

# Finding all image paths, and mapping them to the DataFrame
subfolders = [f"images_{i:03}/images" for i in range(1, 13)]  # Generates 'images_001' to 'images_012'
path_mapping = {}
for subfolder in subfolders:
    full_folder_path = os.path.join(data_path, subfolder)
    for img_file in os.listdir(full_folder_path):
        path_mapping[img_file] = os.path.join(full_folder_path, img_file)

# Update the DataFrame using the mapping
df_train['Full Image Path'] = df_train['Image Filename'].map(path_mapping)
df_val['Full Image Path'] = df_val['Image Filename'].map(path_mapping)
df_test['Full Image Path'] = df_test['Image Filename'].map(path_mapping)

# Move 'Full Image Path' to the front of the DataFrame
cols_train = ['Full Image Path'] + [col for col in df_train.columns if col != 'Full Image Path']
cols_val = ['Full Image Path'] + [col for col in df_val.columns if col != 'Full Image Path']
cols_test = ['Full Image Path'] + [col for col in df_test.columns if col != 'Full Image Path']
df_train = df_train[cols_train]
df_val = df_val[cols_val]
df_test = df_test[cols_test]

# Drop 'Image Filename' column
train_df = df_train.drop(columns=['Image Filename'])
val_df = df_val.drop(columns=['Image Filename'])
test_df = df_test.drop(columns=['Image Filename'])

# Create class weights
df_train_calculate_weights = df_train.drop(columns=['Full Image Path']).to_numpy()
class_weights_dict = generate_class_weights(df_train_calculate_weights, multi_class=False, one_hot_encoded=True)
class_weights_list = [class_weights_dict[i] for i in class_weights_dict]
class_weights_tensor = torch.tensor(class_weights_list, dtype=torch.float32)

label_weights_dict = {labels[i]: class_weights_dict[i] for i in range(len(labels))}
print(f"Class weights: {label_weights_dict}")

print(f"Train dataframe shape: {df_train.shape} (1 size larger than expected due to 'Full Image Path')")
print(f"Train columns: {df_train.columns}")
print(f"Labels: {labels}")
print(f"Number of labels: {len(labels)}")


Class weights: {'Atelectasis': 0.06666666666666667, 'Cardiomegaly': 0.6968494101318529, 'Effusion': 3.3449700199866754, 'Infiltration': 0.6563995293502418, 'Mass': 0.41714855433698905, 'Nodule': 1.4048125349748182, 'Pneumonia': 1.2366502463054188, 'Pneumothorax': 6.52051948051948, 'Consolidation': 2.237433155080214, 'Edema': 2.024516129032258, 'Emphysema': 4.1494214876033055, 'Fibrosis': 4.01664, 'Pleural_Thickening': 4.63601108033241, 'Hernia': 2.6055007784120394}
Train dataframe shape: (75312, 16) (1 size larger than expected due to 'Full Image Path')
Train columns: Index(['Full Image Path', 'Image Filename', 'Atelectasis', 'Cardiomegaly',
       'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
       'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis',
       'Pleural_Thickening', 'Hernia'],
      dtype='object')
Labels: ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphys

## load dataset and transforms

In [5]:
img_size = 224
num_workers = 4
pin_memory = False
batch_size = 32

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR, antialias=True),
    transforms.CenterCrop(img_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=7),
    transforms.ToTensor(),
    normalize,
])

val_transforms = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR, antialias=True),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    normalize,
])

In [7]:
train_dataset = ChestXray14HFDataset(
    dataframe=train_df, transform=train_transforms)
val_dataset = ChestXray14HFDataset(
    dataframe=val_df, transform=val_transforms)
test_dataset = ChestXray14HFDataset(
    dataframe=test_df, transform=val_transforms)

In [8]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)

## trainer

In [9]:
class MultiLabelInferenceModel(LightningModule):
    def __init__(self, model, criterion, learning_rate, num_labels, img_size, labels):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.num_labels = num_labels
        self.log_step_interval = 100
        self.img_size = img_size
        self.labels = labels

        self.f1_score = MultilabelF1Score(num_labels=self.num_labels, threshold=0.5, average='macro')
        self.f1_score_micro = MultilabelF1Score(num_labels=self.num_labels, threshold=0.5, average='micro')
        self.auroc = AUROC(task="multilabel", num_labels=self.num_labels, average="macro")
        self.auroc_classwise = AUROC(
            task="multilabel",
            num_labels=self.num_labels,
            average=None,
        )


    def forward(self, pixel_values):
        return self.model(pixel_values)

    def step(self, batch):
        pixel_values = batch['pixel_values']
        labels = batch['labels']

        logits = self.forward(pixel_values)

        loss = self.criterion(logits, labels)
        return loss, logits, labels

    def training_step(self, batch, batch_idx):
        loss, logits, labels = self.step(batch)
        self.log('train_loss', loss, on_step=True,
                 on_epoch=True, prog_bar=True, logger=True)
        # Update metrics
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)
        auroc_classwise = self.auroc_classwise_with_sigmoid(logits, labels.type(torch.int32))

        if batch_idx % self.log_step_interval == 0:
            self.calc_classwise_auroc(auroc_classwise)
            self.log('train_f1', f1, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_f1_micro', f1_micro, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_auroc', auroc, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_loss', loss, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):

        loss, logits, labels = self.step(batch)
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)
        auroc_classwise = self.auroc_classwise_with_sigmoid(logits, labels.type(torch.int32))

        if batch_idx % self.log_step_interval == 0:
            self.calc_classwise_auroc(auroc_classwise)
            self.log('val_f1', f1, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('val_f1_micro', f1_micro, on_step=True,
                        on_epoch=True, prog_bar=True, logger=True)
            self.log('val_auroc', auroc, on_step=True,
                        on_epoch=True, prog_bar=True, logger=True)
            self.log('val_loss', loss, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)

    def calc_classwise_auroc(self, auroc_classwise):
        for label_name, score in zip(self.label_names, auroc_classwise):
            self.log(f'train_auroc_{label_name}', score, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        loss, logits, labels = self.step(batch)
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)
        auroc_classwise = self.auroc_classwise_with_sigmoid(logits, labels.type(torch.int32))

        self.calc_classwise_auroc(auroc_classwise)
        self.log('test_loss', loss)
        self.log('test_f1', f1)
        self.log('test_f1_micro', f1_micro)
        self.log('test_auroc', auroc)

        return {'test_loss': loss, 'test_f1': f1, 'test_f1_micro': f1_micro}

    def on_test_end(self):
        self.save_model()

    def configure_optimizers(self):
        optimizer = self.optimizer
        scheduler = self.scheduler
        return [optimizer], [scheduler]

    def save_model(self):
        self.file_manager.logger.info(onnx.__version__)  # This will print the version of ONNX installe

        self.to_onnx(f"./test-model.onnx", input_sample=torch.randn(1, 3, self.img_size, self.img_size))

    def f1_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.f1_score(preds, labels)

    def f1_micro_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.f1_score_micro(preds, labels)

    def auroc_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.auroc(preds, labels.type(torch.int32))

    def auroc_classwise_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.auroc_classwise(preds, labels.type(torch.int32))

## load model

In [22]:
model_file="/cluster/home/taheeraa/code/master-thesis/01-multi-label/checkpoints/DenseNet121_aug4_pretrain_WeightBelow1_1_0.829766922537.pkl"

In [63]:
from torchvision.models import resnet50, resnet34, alexnet, vit_b_16, densenet121, efficientnet_b1
import torch.nn as nn

def classifying_head(in_features: int, num_labels: int):
    return nn.Sequential(
        nn.Linear(in_features=in_features, out_features=512),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(in_features=512, out_features=256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.BatchNorm1d(num_features=256),
        nn.Linear(256, num_labels),
    )

model = efficientnet_b1(weights="IMAGENET1K_V1")
#model = resnet50(weights='IMAGENET1K_V2')
img_size = int(224) 

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Get the number of parameters
num_params = count_parameters(model)
print(f'before: The model has {num_params} trainable parameters')

model.classifier = classifying_head(1408, 14)
num_params = count_parameters(model)
print(f'after: The model has {num_params} trainable parameters')

before: The model has 7794184 trainable parameters
after: The model has 7370030 trainable parameters
